In [ ]:
import streamlit as st
from langgraph.graph import StateGraph
from typing import TypedDict, Literal
import fitz  # PyMuPDF
from docx import Document
import tempfile

# ======= PROMPTS & UTILS =======
summarizer_prompt = """
너는 자료를 읽고 해당 자료로부터 주요내용을 추출하는 거야  
가장 추출할 수 있을 만큼 압축해서 내용을 추출해주는데

3가지는 반드시 가지고 있어야해.

1) 기법 설명  
2) 장점 및 효과  
3) 예상 질문  
핵심 내용만 담은 상태로 가장 압축시켜서  
a4 용지 1장 이내로 정리해줘

빨강색 밑줄 글자로 논문 제목 별 나뉘어져 있어
"""

critic_prompt = """
다음은 한 문서와 그것을 요약한 내용이야.  
요약이 아래 조건을 충족하는지 평가해줘:

1. 기법 설명이 구체적이고 간결하게 요약되었는가?  
2. 장점 및 효과가 논리적으로 도출되었는가?  
3. 예상 질문이 타당하고 실제 질문 가능성이 높은가?

이 3가지 항목 각각에 대해 ✅ 또는 ❌로 체크해주고, ❌가 있다면 그 이유를 구체적으로 설명해줘.  
그리고 "수정 필요" 또는 "완료" 중 하나를 최종 판단으로 내려줘.
"""

def extract_text_from_pdf(file) -> str:
    text = ""
    with fitz.open(stream=file.read(), filetype="pdf") as doc:
        for page in doc:
            text += page.get_text()
    return text

def save_summary_to_word(summary: str) -> str:
    doc = Document()
    doc.add_heading("📄 논문 요약", 0)
    doc.add_paragraph(summary)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".docx")
    doc.save(temp_file.name)
    return temp_file.name

def fake_llm(prompt: str) -> str:
    if "평가해줘" in prompt:
        return "1. ✅\n2. ✅\n3. ✅\n\n최종 판단: 완료"
    return "[요약 예시] 기법은 Self-Para-Consistency를 활용해 샘플링 수를 줄이는 전략이고..."

# ======= MULTI-AGENT LOGIC =======
class State(TypedDict):
    document: str
    summary: str
    feedback: str
    status: Literal["in_progress", "done"]
    loop_count: int

def call_summarizer(document: str, feedback: str = "") -> str:
    prompt = summarizer_prompt
    if feedback:
        prompt += f"\n\nCritic의 피드백을 참고하여 수정:\n{feedback}"
    return fake_llm(prompt + "\n\n" + document)

def call_critic(document: str, summary: str):
    prompt = critic_prompt + f"\n\n문서:\n{document}\n\n요약:\n{summary}"
    critique = fake_llm(prompt)
    if "완료" in critique:
        return critique, "done", ""
    else:
        feedback = critique.split("❌")[1].strip() if "❌" in critique else critique
        return critique, "in_progress", feedback

def summarizer_node(state):
    summary = call_summarizer(state["document"], state.get("feedback", ""))
    return {**state, "summary": summary, "loop_count": state["loop_count"] + 1}

def critic_node(state):
    critique, status, feedback = call_critic(state["document"], state["summary"])
    st.markdown(f"### 🧐 Critic 평가 ({state['loop_count']}회차)")
    st.code(critique)
    return {**state, "feedback": feedback, "status": status}

def check_continue(state):
    if state["status"] == "done" or state["loop_count"] >= 3:
        return "exit"
    return "continue"

def build_graph():
    builder = StateGraph(State)
    builder.add_node("summarizer", summarizer_node)
    builder.add_node("critic", critic_node)
    builder.set_entry_point("summarizer")
    builder.add_edge("summarizer", "critic")
    builder.add_conditional_edges("critic", check_continue, {
        "continue": "summarizer",
        "exit": None
    })
    return builder.compile()


2025-04-01 23:37:27.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:37:27.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:37:27.637 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:37:27.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:37:27.638 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:37:27.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:37:27.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-01 23:37:27.640 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar